<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Isochrone-map-using-OSM-data" data-toc-modified-id="Isochrone-map-using-OSM-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Isochrone map using OSM data</a></span><ul class="toc-item"><li><span><a href="#Isochrone-map" data-toc-modified-id="Isochrone-map-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Isochrone map</a></span></li><li><span><a href="#Querying-a-places'-data" data-toc-modified-id="Querying-a-places'-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Querying a places' data</a></span></li><li><span><a href="#Calculations" data-toc-modified-id="Calculations-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Calculations</a></span></li><li><span><a href="#Colouring:-from-point-to-area" data-toc-modified-id="Colouring:-from-point-to-area-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Colouring: from point to area</a></span></li><li><span><a href="#Shape-the-colouring" data-toc-modified-id="Shape-the-colouring-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Shape the colouring</a></span><ul class="toc-item"><li><span><a href="#Exercise" data-toc-modified-id="Exercise-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Exercise</a></span></li></ul></li></ul></li></ul></div>

# Isochrone map using OSM data
OSM data can also be analysed for information as for example speed and travel time. The following code is mainly done by Geoff Boeing this is one of his examples, we have adapted it to Aachen. He developed OSMnx!  
## Isochrone map
This sort of map despicts the area accessible from a point within a certain time threshold (iso= equal, chrone= time). It is common in geography and urban plannung, and we can create one using OSM data and Python package OSMnx. But let us start with importing the necessary libraries.

In [ ]:
import sys
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from descartes import PolygonPatch
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon

%matplotlib inline
ox.config(log_console=True)
ox.__version__

## Querying a places' data
The function graph_from_place can only be used for areas where the query must be geocodable and OSM must have polygon boundaries for result. Alternatively use graph_from_adress function. Here, the geocode converts the place name to a point.

In [ ]:
#Set needed values for parameter
place = "Templergraben 55, Aachen, Germany"
network_type = "walk"
dist = 1000
trip_times = [5, 10, 15, 20]
travel_speed = 4.5 #walking speed in km/hour

In [ ]:
G = ox.graph_from_address(place, dist=dist, network_type=network_type)

In [ ]:
#Find the centermost node and then project the graph to UTM
gdf_nodes = ox.graph_to_gdfs(G, edges=False)
x, y = gdf_nodes["geometry"].unary_union.centroid.xy
center_node = ox.nearest_nodes(G, x[0], y[0])
G = ox.project_graph(G)

## Calculations

In [ ]:
#Add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
for _, _, _, data in G.edges(data=True, keys=True):
    data["time"] = data["length"] / meters_per_minute

In [ ]:
#Get the color
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap="plasma", start=0, return_hex=True)

In [ ]:
#Color the nodes according to isochrone then plot the street network
node_colors = {}
for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")
    for node in subgraph.nodes():
        node_colors[node] = color
#Define node color and size
nc = [node_colors[node] if node in node_colors else "none" for node in G.nodes()]
ns = [15 if node in node_colors else 0 for node in G.nodes()]
#Plotting the graph
fig, ax = ox.plot_graph(
    G,
    node_color=nc,
    node_size=ns,
    node_alpha=0.8,
    edge_linewidth=0.3,
    edge_color="#ffffff",
)
fig.savefig('plot.png', dpi=300)

## Colouring: from point to area
So until now, we have got coloured points, but we would like to have polygons in our map. So what we do is, we take the nodes and calculate the polygon.

In [ ]:
#Create the isochrone polygons
isochrone_polys = []
for trip_time in sorted(trip_times, reverse=True):
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")
    node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)

In [ ]:
#Create graph plot
fig, ax = ox.plot_graph(
    G, show=False, close=False, edge_color="#ffffff", edge_alpha=0.2, node_size=0
)
#Adding polygons
for polygon, fc in zip(isochrone_polys, iso_colors):
    patch = PolygonPatch(polygon, fc=fc, ec="none", alpha=0.6, zorder=-1)
    ax.add_patch(patch)
plt.show()

## Shape the colouring
The next part is huge and contains mathematic. It is the first time we actually create a function. We want the polygon we already got to follow the street shape, therefore we need to do some reshaping. Go through the following code and try to understand what happens.

In [ ]:
#Creating a function for polygon shaping
def make_iso_polys(G, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")
        node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index("id")

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", LineString([f, t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union
        if infill:
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys.append(new_iso)
    return isochrone_polys

In [ ]:
isochrone_polys = make_iso_polys(G, edge_buff=25, node_buff=0, infill=True)
#Create the graph plot
fig, ax = ox.plot_graph(
    G, show=False, close=False, edge_color="#ffffff", edge_alpha=0.2, node_size=0
)
#Adding polygons
for polygon, fc in zip(isochrone_polys, iso_colors):
    patch = PolygonPatch(polygon, fc=fc, ec="none", alpha=0.7, zorder=-1)
    ax.add_patch(patch)
plt.show()

### Exercise
Center your own living address and find out what is reachable using only bike network. Figure out what the average speed using a bike is and assume trip times in a range from 20 to 50 and use an other colour map.